In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD")
dbutils.widgets.text("target_table", " ")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","VCM_DWH_PRD.D_STORE, VCM_DWH_PRD.D_PRODUCT"
)
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")


In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_min_stock
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
) AS
WITH
min_stock AS (
    SELECT
        b.store_id,
        c.product_id AS article,
        c.base_uom,
        CASE WHEN d.order_method = 'Centralize' THEN 'Eton' ELSE 'VCM' END AS order_method,
        CASE WHEN d.order_method = 'Centralize' THEN 'Giao DC' ELSE 'Giao TT' END AS delivery_method,
        a.proc_date,
        SUM(a.qty) AS min_stock,
        NULL AS asst,
        'VM' AS source,
        start_date,
        end_date
    FROM {catalog_name}.udp_wcm_silver_dim.stg_minimun_stock_for_dr_vm a
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.size_st = b.size_st AND business_unit = '1500'
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product c ON a.mch5 = c.mch5_id
    LEFT JOIN {catalog_name}.udp_wcm_silver_dim.sitemaster d ON b.store_id = d.store_id
    LEFT JOIN {catalog_name}.udp_wcm_silver_dim.vm_product_flow_to_dh e 
        ON b.store_id = e.store_id
        AND c.product_id = e.product_id
    GROUP BY 1,2,3,4,5,6,9,10,11

    UNION ALL

    SELECT
        store_id,
        article,
        base_uom,
        order_method,
        delivery_method,
        proc_date,
        min_stock,
        asst,
        'VMP' AS source,
        start_date,
        end_date
    FROM {catalog_name}.udp_wcm_silver_dim.stg_min_stock_for_vmp

    UNION ALL

    SELECT
        store_id,
        article,
        base_uom,
        order_method,
        delivery_method,
        CAST(proc_date AS DATE) AS proc_date,
        min_stock,
        asst,
        'KVI' AS source,
        CAST('2020-09-22' AS DATE) AS start_date,
        CAST('2400-01-01' AS DATE) AS end_date
    FROM {catalog_name}.udp_wcm_silver_dim.min_stock_kvi_fresh
    WHERE proc_date IN (
        SELECT MAX(proc_date)
        FROM {catalog_name}.udp_wcm_silver_dim.min_stock_kvi_fresh
    )
),

ranked AS (
    SELECT *,
        ROW_NUMBER() OVER (PARTITION BY store_id, article, end_date ORDER BY proc_date DESC) AS rn
    FROM min_stock
),

main AS (
    SELECT 
        a.* EXCEPT(rn, proc_date, source),
        IFNULL(b.source, a.source) AS source
    FROM ranked a
    LEFT JOIN (
        SELECT DISTINCT 
            store_id,
            article,
            'KVI' AS source
        FROM {catalog_name}.udp_wcm_silver_dim.min_stock_kvi_fresh
        WHERE proc_date IN (
            SELECT MAX(proc_date)
            FROM {catalog_name}.udp_wcm_silver_dim.min_stock_kvi_fresh
        )
    ) b ON a.store_id = b.store_id AND a.article = b.article
    WHERE rn = 1
),

period AS (
    SELECT DISTINCT start_date, end_date, source
    FROM main
),

last_period AS (
    SELECT start_date, end_date, source,
        LAG(start_date, 1) OVER (PARTITION BY source ORDER BY start_date) AS last_period
    FROM period
),

main1 AS (
    SELECT *,
        LAG(m.start_date, 1) OVER (PARTITION BY store_id, article ORDER BY m.start_date) AS day_of_sale
    FROM main m
)

SELECT m.*,
    IF(m.day_of_sale = l.last_period, l.last_period, NULL) AS last_period
FROM main1 m
LEFT JOIN last_period l
    ON m.start_date = l.start_date AND m.source = l.source
""")